In [1]:
from gmail_api import init_gmail_service, get_email_messages, get_email_message_details

client_file = 'credentials.json'
service = init_gmail_service(client_file)

gmail v1 service created successfully


In [2]:
messages = get_email_messages(service,max_results=5)
messages

[{'id': '195e85483e630053', 'threadId': '195e85483e630053'},
 {'id': '195e84fced5aba16', 'threadId': '195e84fced5aba16'},
 {'id': '195e8395349a3fe3', 'threadId': '195e8395349a3fe3'},
 {'id': '195e8259744748d9', 'threadId': '195e8259744748d9'},
 {'id': '195e8163b4947f60', 'threadId': '195e8163b4947f60'}]

In [4]:
for msg in messages:
    details = get_email_message_details(service,msg['id'])
    if details:
        print(details)

{'subject': 'It’s go time on savings 🏀', 'sender': 'Grubhub <email@a.grubhub.com>', 'recipients': 'vaidhdhruv@gmail.com', 'body': "Tune in, score big, repeat  \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n  Grubhub | Elite Eats Only \r\n( http://ablinksuni.a.grubhub.com/ss/c/u001.pJ-ifDyEIbMPOy1oXfEp38KSUw2KkRnYnHPnJd4vNGmLAeUEOp1rswf8Pi3JYY5aDgPCXd0P7keXoC5wnFsR3I-l1Kwf67k52s_g26AOz-gUjrZ1Vr_IQSOHWAcqcsi19gAiW4xmda-1zC37IICRQ-apaRqe17SDN5vP2gd_S0AIxPiRE2sG2RY6bq4G9PVLWtOw_m-9dn0EYMopfM-hK3xh482dMEvnmxvsBT1UQgsRqdDsnRnUUGp-vQQ-rZRcUaRSjlylmamp3OJmPZt5_DPwx7X27Vv8HmLkdT3RxyLhvXjx27cXL5Kae5_3RlOND0Q3HpoJJIYkfszLTcMtXW12T_fkbLxD9fnDCrkBk4wGnwuQ7Rsp5JTw2Gos9a2t/4f8/EMV3CJ8TSZKpBie7kzpQ2A/t0/h001.-yIoS9hWezXAiJM6wqbbMltEYJUrH-c3Vwz6eG1Ejag )\r\n\r\n\r\n*****************************************************************\r\nGet fan-favorite foods delivered to fuel up as you cheer your\r\nteam on through the final rounds of the tournament. It’s go time!\r\n*****************************************************